Creating an API connection to the FDA and pulling in the data to MongoDB

In [48]:
import requests
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
# import plotly.express as px
# from geopy.geocoders import Nominatim
# import time
# import plotly.graph_objects as go
# import seaborn as sns
# import matplotlib.pyplot as plt
# from ipywidgets import widgets
# from IPython.display import display, clear_output
# import ipywidgets as widgets
# from ipywidgets import interact

# Define the API endpoint and parameters
url = "https://api.fda.gov/food/enforcement.json?search=report_date:[20200101+TO+20241201]&limit=1000"

# Send a GET request to the API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    recalls = data.get('results', [])
    recall_df = pd.DataFrame(recalls)

    # Convert DataFrame to dictionary format for MongoDB
    recall_records = recall_df.to_dict("records")

    # Establish a connection to MongoDB
    client = MongoClient('mongodb://localhost:27017/')  # Replace with your MongoDB connection string

    # Access a database and collection
    db = client["fda_data"]  # Replace "fda_data" with your desired database name
    collection = db["food_recalls"]  # Replace "food_recalls" with your desired collection name

    # Insert modified data into MongoDB
    collection.insert_many(recall_records)

else:
    print(f"Failed to retrieve data: {response.status_code}")


In [49]:
# Checking number of recoreds pulled in from the FDA
num_records = len(recall_records)
num_records 

1000

Cleaning & Formating Data in MongoDB

In [ ]:
####### CLEANING, FILTERING COLUMNS & HEADERS #######

# Finding and droping all NaNs from the "termination_date" column 
collection.find({"termination_date": {"$in": [None, "", float("nan")]}})
collection.delete_many({"termination_date": {"$in": [None, float("nan")]}})

# Filter by 'United States' only and remove all other Countries
collection.delete_many({"country": {"$ne": "United States"}})

# Removing the columns not needed
collection.update_many({}, {"$unset": {"openfda": "",
                                       "event_id": "", 
                                       "product_type": "",
                                       "address_2": "",                                           
                                       "voluntary_mandated": "",
                                       "initial_firm_notification": "",
                                       "distribution_pattern": "",
                                       "recall_number": "",
                                       "product_description": "",
                                       "product_quantity": "",
                                       "reason_for_recall": "",
                                       "report_date": "",
                                       "code_info": "",
                                       "more_code_info": ""
                                       }}          
)

# Renaming the columns
collection.update_many({}, {"$rename": {"status": "Status",
                                        "city": "City",
                                        "state": "State",
                                        "country": "Country",
                                        "classification": "Classification",
                                        "recalling_firm": "Recall Firm",
                                        "address_1": "Address",
                                        "postal_code": "Postal Code",
                                        "recall_initiation_date": "Recall Initiation Date",
                                        "center_classification_date": "Center Classification Date",
                                        "termination_date": "Termination Date"
                                        }}
)

####### FORMATING DATE COLUMNS #######

for date_field in ["Center Classification Date", "Recall Initiation Date", "Termination Date"]:
    documents = collection.find({date_field: {"$exists": True}})
    for doc in documents:
        date_str = doc.get(date_field)
        
        # Check if the date_str is a valid string and is 8 digits long
        if isinstance(date_str, str) and len(date_str) == 8 and date_str.isdigit():
            try:
                formatted_date = datetime.strptime(date_str, "%Y%m%d")
                collection.update_one(
                    {"_id": doc["_id"]},
                    {"$set": {date_field: formatted_date}}
                )
            except ValueError as e:
                print(f"Error parsing date for document ID {doc['_id']}: {e}")
        else:
            # Log or handle cases where date_str is not a valid string or is not 8 digits long
            print(f"Skipping invalid date for document ID {doc['_id']}: {date_str}")


####### CALCULATING DAYS FOR CLASSIFICATION #######

# Adding new column for days it to to classify and calculating its value
new_column = "Days To Classify"

for document in collection.find():
    start_date = document.get("Recall Initiation Date")
    end_date = document.get("Center Classification Date")
    
    if start_date and end_date:
        new_column_value = (end_date - start_date).days
        collection.update_one(
            {"_id": document["_id"]},
            {"$set": {"Days To Classify": new_column_value}}
        )

collection.find_one()

{'_id': ObjectId('674f2dc196072a4e67166562'),
 'Address': '2610 Homestead Pl',
 'Center Classification Date': datetime.datetime(2020, 4, 13, 0, 0),
 'City': 'Rancho Dominguez',
 'Classification': 'Class III',
 'Country': 'United States',
 'Postal Code': '90220-5610',
 'Recall Firm': 'Organic By Nature, Inc.',
 'Recall Initiation Date': datetime.datetime(2020, 2, 24, 0, 0),
 'State': 'CA',
 'Status': 'Terminated',
 'Termination Date': datetime.datetime(2021, 2, 2, 0, 0),
 'Days To Classify': 49}

In [53]:
# Query data back from MongoDB
recalls_from_db = list(collection.find())

# Convert back to DataFrame if needed
db_df = pd.DataFrame(recalls_from_db)

#unique_values = db_df['_id'].unique()
#unique_values = db_df['Country'].unique()
#unique_values = db_df['Days To Classify'].unique()
#unique_values = db_df['Termination Date'].unique()
#unique_values = db_df['Classification'].unique()

#unique_values

# Checking number of recoreds remaining after clean up
clean_records = len(db_df)
clean_records

848